## Expedited patterns processing
This notebook reads in the patterns data and:
    1. Filters down to Philadelphia zipcodes based on a (currently hard-coded) list.
    2. Aggregates each file by zipcode
    3. Concatenates the files
    4. Writes the result to philly_patterns_by_zip.csv in the processed data folder.
    
It runs much faster than the original patterns exploration notebook which makes it useful 
for updating the zipcode map data.

In [1]:
import pandas as pd
from safegraph_py_functions import safegraph_py_functions as sgpy
import os
from dotenv import load_dotenv, find_dotenv
from loguru import logger

In [2]:
pd.options.display.max_columns = 999

In [3]:
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)
os.chdir(os.environ.get("ROOT_DIR"))
from src import DATA_DIR
raw_data_dir = DATA_DIR / 'raw'

In [4]:
# Read in all patterns files in the monthly-patterns folder

patterns_path = raw_data_dir / "monthly-patterns-2020-12"

files = []
for f in patterns_path.glob("**/*.csv.gz"):
    files.append(f)

In [6]:
# columns that we keep and aggregate
keep_cols = ["postal_code", "safegraph_place_id", "date_range_start", "visits_by_day"]
# columns we keep after exploding
keep_cols_2 = ["postal_code", "date", "day_visit_counts"]
# columns to group by
group_by_cols = ["postal_code", "date"]


def filter_to_philly(df):
    # zip codes are read as integers rather than strings so we add leading zeros.
    # this is not strictly necessary since Philadelphia zipcodes don't have leading zeros.

    # Philadelphia selection
    # HK: adding leading zeros because some zipcodes in MA are 0191X.
    df["postal_code"] = df["postal_code"].apply(lambda x: ("00000" + str(x))[-5:])
    in_philly = df["postal_code"].astype(str).str.startswith("191")
    df = df.loc[in_philly]

    return df


def explode(df):

    df = df[keep_cols]

    # The visits_by_day column contains a list of integers.
    # This explodes that list so we get one row per day.
    df = sgpy.explode_json_array(
        df,
        array_column="visits_by_day",
        value_col_name="day_visit_counts",
        place_key="safegraph_place_id",
        file_key="date_range_start",
        array_sequence="day",
        keep_index=False,
        zero_index=False,
    )
    df["date_range_start"] = pd.to_datetime(df["date_range_start"])
    # Calculate the date for each row.
    temp = df["day"].apply(lambda x: pd.Timedelta(x - 1, unit="D"))
    df["date"] = df["date_range_start"] + temp
    df = df[keep_cols_2].groupby(group_by_cols).agg("sum").reset_index()
    return df

In [7]:
philly_patterns = []
for i, f in enumerate(files):
    print(f)
    philly_patterns.append(filter_to_philly(pd.read_csv(f)))
    
philly_patterns = pd.concat(philly_patterns)

/Users/hannahkronenberg/SafegraphCOVIDPhilly/src/../data/raw/monthly-patterns-2020-12/patterns/2021/03/06/02/patterns-part3.csv.gz
/Users/hannahkronenberg/SafegraphCOVIDPhilly/src/../data/raw/monthly-patterns-2020-12/patterns/2021/03/06/02/patterns-part1.csv.gz
/Users/hannahkronenberg/SafegraphCOVIDPhilly/src/../data/raw/monthly-patterns-2020-12/patterns/2021/03/06/02/patterns-part4.csv.gz
/Users/hannahkronenberg/SafegraphCOVIDPhilly/src/../data/raw/monthly-patterns-2020-12/patterns/2021/03/06/02/patterns-part2.csv.gz
/Users/hannahkronenberg/SafegraphCOVIDPhilly/src/../data/raw/monthly-patterns-2020-12/patterns/2021/04/08/06/patterns-part3.csv.gz
/Users/hannahkronenberg/SafegraphCOVIDPhilly/src/../data/raw/monthly-patterns-2020-12/patterns/2021/04/08/06/patterns-part1.csv.gz
/Users/hannahkronenberg/SafegraphCOVIDPhilly/src/../data/raw/monthly-patterns-2020-12/patterns/2021/04/08/06/patterns-part4.csv.gz
/Users/hannahkronenberg/SafegraphCOVIDPhilly/src/../data/raw/monthly-patterns-2020-

In [8]:
philly_patterns_clean = philly_patterns.loc[philly_patterns["visits_by_day"].notnull()]

In [20]:
philly_patterns_clean.head()

,placekey,safegraph_place_id,parent_placekey,parent_safegraph_place_id,location_name,street_address,city,region,postal_code,safegraph_brand_ids,brands,date_range_start,date_range_end,raw_visit_counts,raw_visitor_counts,visits_by_day,poi_cbg,visitor_home_cbgs,visitor_daytime_cbgs,visitor_country_of_origin,distance_from_home,median_dwell,bucketed_dwell_times,related_same_day_brand,related_same_month_brand,popularity_by_hour,popularity_by_day,device_type,top_category,sub_category,category_tags,naics_code,latitude,longitude,iso_country_code,phone_number,open_hours,opened_on,closed_on,tracking_opened_since,tracking_closed_since
59,22m-223@628-p8h-nbk,sg:05ede5a3192b44638dcd954d369b4856,NaN,NaN,Save-A-Lot,6801 Frankford Ave,Philadelphia,PA,19135,SG_BRAND_c5a2c9092cad648bc1ee9c9be08f83ab,Save-A-Lot,2021-02-01T00:00:00-05:00,2021-03-01T00:00:00-05:00,164.0,127.0,"[3,8,4,4,6,6,6,13,5,5,3,6,5,6,3,10,6,5,3,11,6,...",4.210103e+11,"{""420912019014"":5,""421010315012"":5,""4210103150...","{""421010319005"":6,""421010349001"":5,""4210103150...","{""US"":121}",1408.0,17.0,"{""<5"":11,""5-10"":50,""11-20"":38,""21-60"":56,""61-1...","{""Planet Fitness"":16,""Petco"":10}","{""ShopRite"":53,""Wawa"":48,""Dunkin'"":42,""Dollar ...","[0,0,0,0,0,3,2,8,6,12,15,20,30,22,20,17,25,14,...","{""Monday"":25,""Tuesday"":31,""Wednesday"":19,""Thur...","{""android"":82,""ios"":44}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250,22b-222@628-p7b-vmk,sg:1501a801e04b4615b1af620ccf1a2e48,NaN,NaN,CARQUEST Auto Parts,3093 Holme Ave,Philadelphia,PA,19136,SG_BRAND_ed5a1ef3e85b774022eb505cf6f184ee,CARQUEST Auto Parts,2021-02-01T00:00:00-05:00,2021-03-01T00:00:00-05:00,2.0,2.0,"[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0...",4.210103e+11,{},{},{},NaN,29.0,"{""<5"":0,""5-10"":1,""11-20"":0,""21-60"":1,""61-120"":...",{},"{""Wawa"":99,""Dunkin'"":97,""The Fresh Works"":50,""...","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0]","{""Monday"":0,""Tuesday"":0,""Wednesday"":0,""Thursda...","{""android"":0,""ios"":0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
334,222-222@628-p9k-y9z,sg:1d184d77616941d2afa14b4d3831d7e4,NaN,NaN,Extra Space Storage,11501 Roosevelt Blvd,Philadelphia,PA,19154,SG_BRAND_25f99d0cc5d6078042c8f466f6a8fa83,Extra Space Storage,2021-02-01T00:00:00-05:00,2021-03-01T00:00:00-05:00,27.0,18.0,"[2,0,1,1,0,1,2,0,1,0,1,0,0,1,1,2,0,1,0,2,1,3,3...",4.210198e+11,"{""421010301004"":4,""421010336005"":4,""4209120150...","{""421010345011"":4,""420171014033"":4,""4210103530...","{""US"":17}",6462.0,15.0,"{""<5"":0,""5-10"":9,""11-20"":9,""21-60"":6,""61-120"":...",{},"{""Wawa"":66,""Dunkin'"":63,""CVS"":45,""Sunoco"":42,""...","[1,2,2,1,0,0,0,0,1,3,6,2,4,2,2,3,4,6,2,0,0,0,0,1]","{""Monday"":6,""Tuesday"":6,""Wednesday"":1,""Thursda...","{""android"":12,""ios"":6}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,22j-222@628-pgj-3yv,sg:359580c4d0c24a3aae50813992af8b35,NaN,NaN,La Cienaga Deli Grocery,601 Cecil B Moore Ave,Philadelphia,PA,19122,NaN,NaN,2021-02-01T00:00:00-05:00,2021-03-01T00:00:00-05:00,12.0,11.0,"[0,1,0,0,0,1,0,0,0,0,0,0,1,1,2,1,0,0,1,1,1,0,0...",4.210101e+11,"{""060290028132"":4}","{""060371012101"":4}","{""US"":6}",16608.0,119.0,"{""<5"":0,""5-10"":2,""11-20"":2,""21-60"":1,""61-120"":...",{},"{""Brightside Academy"":9,""United Refrigeration""...","[1,1,1,3,3,6,5,4,3,3,2,2,2,1,1,0,0,0,0,0,0,0,0,1]","{""Monday"":2,""Tuesday"":2,""Wednesday"":0,""Thursda...","{""android"":11,""ios"":0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1057,222-223@628-pzn-zfz,sg:6255ef7faad24ed6a77887c62294abcb,NaN,NaN,Brian Cesar Remax 2000,14500 Bustleton Ave,Philadelphia,PA,19116,NaN,NaN,2021-02-01T00:00:00-05:00,2021-03-01T00:00:00-05:00,3.0,3.0,"[0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0...",4.210104e+11,"{""421010363033"":4}","{""421010363033"":4}",{},NaN,40.0,"{""<5"":0,""5-10"":1,""11-20"":0,""21-60"":1,""61-120"":...",{},"{""Net Cost Market"":33,""Radiator"":33,""Philly Pr...","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2,0,0,

In [21]:
len(philly_patterns_clean)

495464

In [22]:
philly_patterns.to_csv(
    DATA_DIR / "processed" / "philly_patterns.csv.tar.gz", index=False
)

In [23]:
philly_patterns_exploded = explode(philly_patterns_clean)

In [24]:
philly_patterns_exploded.to_csv(
    DATA_DIR / "processed" / "philly_patterns_exploded.csv.tar.gz", index=False
)